In this notebook, we will walk through the process of:

1. Initializing the YouTubeDataClient with your API key.
2. Collecting video metadata from one or multiple channels.
3. Retrieving performance statistics for selected videos.
4. Extracting top-level user comments for a set of videos.

The outputs can be returned as raw dictionaries, Pandas DataFrames, or Polars DataFrames,
allowing you to flexibly integrate them into any data pipeline or workflow.

For demonstration purposes, the [CDCodes channel](https://www.youtube.com/@cdcodes) and [homedawg channel](https://www.youtube.com/@homedawg_yt) will be used, as these belong to one of the developers of this package. You are free to subsitute these for a channel or channels of your choosing

# Setup - Load API Key and YouTubeDataClient

To generate an API key for your google developer account, please see the [official YouTube API v3 documentation](https://developers.google.com/youtube/v3/getting-started) for more information

In [2]:
# Load environment variables
import os
from yt_stats_wrangler.api.client import YouTubeDataClient

# Channel ID(s) to test the package
# Feel free to test the package with those of your choosing
TEST_CHANNEL_ID = 'UCB2mKxxXPK3X8SJkAc-db3A' # https://www.youtube.com/@cdcodes
TEST_CHANNEL_IDS = ['UCB2mKxxXPK3X8SJkAc-db3A', 'UC0pIFIthf92_8ku2NAYFG_Q'] # https://www.youtube.com/@cdcodes, https://www.youtube.com/@homedawg_yt

# Get your API key - recommended to store it as an environment variable
YOUTUBE_API_KEY = os.getenv("YOUTUBE_API_V3_KEY")

if not YOUTUBE_API_KEY:
    raise ValueError("Missing YOUTUBE_API_V3_KEY. it is recommended that you set it as an environment variable.")

# Create the client (with quoata limit set to the free trial of 10000 units)
client = YouTubeDataClient(api_key=YOUTUBE_API_KEY, max_quota=10000)
print(f"Remaining units in quota: {client.max_quota - client.quota_used}")

Remaining units in quota: 10000


# Gather Video IDs and Metadata

Series of functions that can take a YouTube channel ID and gather the following

+ Titles, Descriptions, and upload dates for public videos on a YouTube channle or channels
+ Video IDs that can be passed to other methods to collect descriptive data

## Gather video IDs, titles and descriptions from a single channel

### Raw output from YouTube API V3

In [3]:
# Get all public videos from a channel and print out in raw form
videos = client.get_all_video_details_for_channel(TEST_CHANNEL_ID, key_format="raw", output_format='raw')
print(f"Collected {len(videos)} videos.")
videos[:1]  # Preview first  result
print(f"Remaining units in quota: {client.max_quota - client.quota_used}")

Collected 18 videos.
Remaining units in quota: 9998


### Pandas output with modified column names

In [4]:
# Get all public videos from a channel and output a pandas dataframe with formatted column names
videos_pandas_df = client.get_all_video_details_for_channel(TEST_CHANNEL_ID, key_format="upper", output_format='pandas')
print(f"Collected {videos_pandas_df.shape[0]} videos.")
print(f"Remaining units in quota: {client.max_quota - client.quota_used}")
videos_pandas_df.head(5)  # Preview first few results

Collected 18 videos.
Remaining units in quota: 9996


,CHANNEL_ID,VIDEO_ID,PUBLISHED_AT,TITLE,DESCRIPTION,CHANNEL_TITLE,VIDEO_DETAILS_COMMIT_TIME
0,UCB2mKxxXPK3X8SJkAc-db3A,72Ef65B65JA,2025-02-09T21:01:50Z,Python DeepSeek Tutorial: Installing DeepSeek ...,"In this tutorial, we'll be installing DeepSeek...",CDcodes,2025-03-31 00:38:18.226572
1,UCB2mKxxXPK3X8SJkAc-db3A,S3dlYi7LPk0,2024-08-18T23:14:38Z,Thank You Pygame,"An ode to pygame, the quirky little python mod...",CDcodes,2025-03-31 00:38:18.226572
2,UCB2mKxxXPK3X8SJkAc-db3A,5OoukJNz3YQ,2023-05-30T00:05:58Z,Beginner Python Project From Scratch: Web Scra...,"In this video, I'll be walking you through a r...",CDcodes,2025-03-31 00:38:18.226572
3,UCB2mKxxXPK3X8SJkAc-db3A,ruISjcokmuk,2022-11-09T01:38:55Z,Beginner Python Project From Scratch: Rock Pap...,Follow along with me on replit: https://replit...,CDcodes,2025-03-31 00:38:18.226572
4,UCB2mKxxXPK3X8SJkAc-db3A,Q6CCdCBVypg,2022-04-05T22:45:48Z,Beginner Python Project From Scratch: Tic Tac Toe,Follow along with me on replit: https://replit...,CDcodes,2025-03-31 00:38:18.226572


### Polars output with modified column names

In [5]:
# Get all public videos from a channel and output a pandas dataframe with formatted column names
videos_polars_df = client.get_all_video_details_for_channel(TEST_CHANNEL_ID, key_format="lower", output_format='polars')
print(f"Collected {videos_polars_df.shape[0]} videos.")
print(f"Remaining units in quota: {client.max_quota - client.quota_used}")
videos_polars_df.head(5)  # Preview first few results

Collected 18 videos.
Remaining units in quota: 9994


channel_id,video_id,published_at,title,description,channel_title,video_details_commit_time
str,str,str,str,str,str,str
"""UCB2mKxxXPK3X8SJkAc-db3A""","""72Ef65B65JA""","""2025-02-09T21:01:50Z""","""Python DeepSeek Tutorial: Inst…","""In this tutorial, we'll be ins…","""CDcodes""","""2025-03-31 00:38:19.116919"""
"""UCB2mKxxXPK3X8SJkAc-db3A""","""S3dlYi7LPk0""","""2024-08-18T23:14:38Z""","""Thank You Pygame""","""An ode to pygame, the quirky l…","""CDcodes""","""2025-03-31 00:38:19.116919"""
"""UCB2mKxxXPK3X8SJkAc-db3A""","""5OoukJNz3YQ""","""2023-05-30T00:05:58Z""","""Beginner Python Project From S…","""In this video, I'll be walking…","""CDcodes""","""2025-03-31 00:38:19.116919"""
"""UCB2mKxxXPK3X8SJkAc-db3A""","""ruISjcokmuk""","""2022-11-09T01:38:55Z""","""Beginner Python Project From S…","""Follow along with me on replit…","""CDcodes""","""2025-03-31 00:38:19.116919"""
"""UCB2mKxxXPK3X8SJkAc-db3A""","""Q6CCdCBVypg""","""2022-04-05T22:45:48Z""","""Beginner Python Project From S…","""Follow along with me on replit…","""CDcodes""","""2025-03-31 00:38:19.116919"""


## Gather video IDs, titles and descriptions from multiple channels at a time

### Raw output from YouTube API V3

In [6]:
# Get all public videos from a channel and print out in raw form
videos = client.get_all_video_details_for_channels(TEST_CHANNEL_IDS, key_format="raw", output_format='raw')
print(f"Collected {len(videos)} videos.")
print(f"Remaining units in quota: {client.max_quota - client.quota_used}")
videos[-1]  # Preview tail of the results

Fetching videos for channel: UCB2mKxxXPK3X8SJkAc-db3A
Fetching videos for channel: UC0pIFIthf92_8ku2NAYFG_Q
Collected 20 videos.
Remaining units in quota: 9990


{'channelId': 'UC0pIFIthf92_8ku2NAYFG_Q',
 'videoId': 'vDm8pvsqFY8',
 'publishedAt': '2024-02-14T09:41:13Z',
 'title': 'What Competitive Super Smash Bros Taught Me (About Life and Myself)',
 'description': "Some reflections on personal development that occurred through my journey as an amateur competitor in Super Smash Bros Melee for the Nintendo Gamecube.\n\n\nTimestamps\n00:00:00 - Introduction\n00:01:26 - Lesson 1~ Improvement is a game (and you can level up)\n00:06:10 - Lesson 2~ You're an obstacle in someone else's story\n00:08:45 - Lesson 3~ Losing sucks, detaching is easier\n00:11:04 - Lesson 4~ Win or lose, vulnerability is cool\n00:14:21 - Lesson 5~ Anyone can step up and be a leader\n00:17:08 - Final Remarks\n\nYou should totally play Melee if you don't already, checkout Slippi:\nhttps://slippi.gg/\n\nMusic in order of appearance\nMii Channel (Editing)\nMenu - Super Smash Bros Melee\n7PM - Animal Crossing New Leaf\nIt's Raining Somewhere Else - Toby Fox\nMenu 2 - Super Smash 

### Pandas output with modified column names

In [7]:
# Get all public videos from a channel and output a pandas dataframe with formatted column names
videos_pandas_df = client.get_all_video_details_for_channels(TEST_CHANNEL_IDS, key_format="lower", output_format='pandas')
print(f"Collected {videos_pandas_df.shape[0]} videos.")
print(f"Remaining units in quota: {client.max_quota - client.quota_used}")
videos_pandas_df.tail(5)  # Preview first few results

Fetching videos for channel: UCB2mKxxXPK3X8SJkAc-db3A
Fetching videos for channel: UC0pIFIthf92_8ku2NAYFG_Q
Collected 20 videos.
Remaining units in quota: 9986


,channel_id,video_id,published_at,title,description,channel_title,video_details_commit_time
15,UCB2mKxxXPK3X8SJkAc-db3A,ePiMYe7JpJo,2020-07-27T15:59:04Z,"Pygame Sprite Sheet Tutorial: How to Load, Par...","In this tutorial, we'll be learning all about ...",CDcodes,2025-03-31 00:38:20.006887
16,UCB2mKxxXPK3X8SJkAc-db3A,bmRFi7-gy5Y,2020-07-24T06:33:18Z,Pygame Menu System Tutorial Part 2: Building t...,LINK TO PART 1: https://youtu.be/a5JWrd7Y_14\n...,CDcodes,2025-03-31 00:38:20.006887
17,UCB2mKxxXPK3X8SJkAc-db3A,a5JWrd7Y_14,2020-07-24T06:03:54Z,Pygame Menu System Tutorial Part 1: Game Loops...,LINK TO PART 2: https://www.youtube.com/watch?...,CDcodes,2025-03-31 00:38:20.006887
18,UC0pIFIthf92_8ku2NAYFG_Q,D7YJXgtLqAU,2024-05-05T23:21:21Z,"Nostalgia, Change and Pokemon",Examining my relationship with the greatest ma...,homedawg,2025-03-31 00:38:20.195383
19,UC0pIFIthf92_8ku2NAYFG_Q,vDm8pvsqFY8,2024-02-14T09:41:13Z,What Competitive Super Smash Bros Taught Me (A...,Some reflections on personal development that ...,homedawg,2025-03-31 00:38:20.195383


### Polars output with modified column names

In [8]:
# Get all public videos from a channel and output a pandas dataframe with formatted column names
videos_polars_df = client.get_all_video_details_for_channels(TEST_CHANNEL_IDS, key_format="upper", output_format='polars')
print(f"Collected {videos_polars_df.shape[0]} videos.")
print(f"Remaining units in quota: {client.max_quota - client.quota_used}")
videos_polars_df.tail(5)  # Preview last few results

Fetching videos for channel: UCB2mKxxXPK3X8SJkAc-db3A
Fetching videos for channel: UC0pIFIthf92_8ku2NAYFG_Q
Collected 20 videos.
Remaining units in quota: 9982


CHANNEL_ID,VIDEO_ID,PUBLISHED_AT,TITLE,DESCRIPTION,CHANNEL_TITLE,VIDEO_DETAILS_COMMIT_TIME
str,str,str,str,str,str,str
"""UCB2mKxxXPK3X8SJkAc-db3A""","""ePiMYe7JpJo""","""2020-07-27T15:59:04Z""","""Pygame Sprite Sheet Tutorial: …","""In this tutorial, we'll be lea…","""CDcodes""","""2025-03-31 00:38:20.524686"""
"""UCB2mKxxXPK3X8SJkAc-db3A""","""bmRFi7-gy5Y""","""2020-07-24T06:33:18Z""","""Pygame Menu System Tutorial Pa…","""LINK TO PART 1: https://youtu.…","""CDcodes""","""2025-03-31 00:38:20.524686"""
"""UCB2mKxxXPK3X8SJkAc-db3A""","""a5JWrd7Y_14""","""2020-07-24T06:03:54Z""","""Pygame Menu System Tutorial Pa…","""LINK TO PART 2: https://www.yo…","""CDcodes""","""2025-03-31 00:38:20.524686"""
"""UC0pIFIthf92_8ku2NAYFG_Q""","""D7YJXgtLqAU""","""2024-05-05T23:21:21Z""","""Nostalgia, Change and Pokemon""","""Examining my relationship with…","""homedawg""","""2025-03-31 00:38:20.791971"""
"""UC0pIFIthf92_8ku2NAYFG_Q""","""vDm8pvsqFY8""","""2024-02-14T09:41:13Z""","""What Competitive Super Smash B…","""Some reflections on personal d…","""homedawg""","""2025-03-31 00:38:20.791971"""


# Get Video Statistics

Once you have a list of video IDs, you can retrieve high-level public statistics
for each video, such as 
+ View counts
+ Like counts
+ Comment counts
+ Tags

This is useful for performance analytics and benchmarking.

### Raw output from YouTube API V3

In [14]:
# Let's grab the first 5 video IDs from the previous data
video_ids = [video["videoId"] for video in videos[:5]]

# Get statistics for the videos
video_stats = client.get_video_stats(video_ids, key_format="raw", output_format="raw")
print(f"Retrieved statistics for {len(video_stats)} videos.")
print(f"Remaining units in quota: {client.max_quota - client.quota_used}")
video_stats[0]

Retrieved statistics for 5 videos.
Remaining units in quota: 9976


{'videoId': '72Ef65B65JA',
 'title': 'Python DeepSeek Tutorial: Installing DeepSeek Locally and Interfacing with Python',
 'description': "In this tutorial, we'll be installing DeepSeek onto our local machine, and then interfacing with the LLM through a Python script. I'll also run through a few very basic use cases that can give you an idea of how you might process data and automate tasks through an LLM.\n\nCode and Datasets can be found here: https://github.com/ChristianD37/YoutubeTutorials/tree/master/DeepSeekPython\n\nollama download: https://ollama.com/download\ndeepseek-r1 page: https://ollama.com/library/deepseek-r1\n\n00:00 - Introduction\n00:56 - Installing ollama and DeepSeek locally\n02:05 - Interfacing with DeepSeek through Python\n03:46 - Cleaner Interface Function\n06:54 - Sentiment Analysis on YouTube Comments\n11:24 - Pokemon 3 to 5 sentence Stat Summaries",
 'publishedAt': '2025-02-09T21:01:50Z',
 'channelId': 'UCB2mKxxXPK3X8SJkAc-db3A',
 'channelTitle': 'CDcodes',
 't

### Pandas output with modified column names

In [19]:
# Let's grab the first 5 video IDs from the pandas dataframe
# Get statistics for the videos
video_stats_pandas_df = client.get_video_stats(videos_pandas_df['video_id'][0:5], key_format="lower", output_format="pandas")
print(f"Retrieved statistics for {video_stats_pandas_df .shape[0]} videos.")
print(f"Remaining units in quota: {client.max_quota - client.quota_used}")
video_stats_pandas_df.head()

Retrieved statistics for 5 videos.
Remaining units in quota: 9972


,video_id,title,description,published_at,channel_id,channel_title,tags,category_id,view_count,like_count,comment_count,duration_seconds,definition,is_short,video_stats_commit_time
0,72Ef65B65JA,Python DeepSeek Tutorial: Installing DeepSeek ...,"In this tutorial, we'll be installing DeepSeek...",2025-02-09T21:01:50Z,UCB2mKxxXPK3X8SJkAc-db3A,CDcodes,[],27,4557,93,18,925,hd,False,2025-03-31 00:43:57.347051
1,S3dlYi7LPk0,Thank You Pygame,"An ode to pygame, the quirky little python mod...",2024-08-18T23:14:38Z,UCB2mKxxXPK3X8SJkAc-db3A,CDcodes,[],27,8748,429,37,400,hd,False,2025-03-31 00:43:57.347051
2,5OoukJNz3YQ,Beginner Python Project From Scratch: Web Scra...,"In this video, I'll be walking you through a r...",2023-05-30T00:05:58Z,UCB2mKxxXPK3X8SJkAc-db3A,CDcodes,[],20,1550,68,11,2737,hd,False,2025-03-31 00:43:57.347051
3,ruISjcokmuk,Beginner Python Project From Scratch: Rock Pap...,Follow along with me on replit: https://replit...,2022-11-09T01:38:55Z,UCB2mKxxXPK3X8SJkAc-db3A,CDcodes,[],20,3323,56,9,960,hd,False,2025-03-31 00:43:57.347051
4,Q6CCdCBVypg,Beginner Python Project From Scratch: Tic Tac Toe,Follow along with me on replit: https://replit...,2022-04-05T22:45:48Z,UCB2mKxxXPK3X8SJkAc-db3A,CDcodes,[],20,78971,811,60,1014,hd,False,2025-03-31 00:43:57.347051


### Polars output with modified column names

In [ ]:
# Let's grab the first 5 video IDs from the pandas dataframe
# Get statistics for the videos
video_stats_polars_df = client.get_video_stats(videos_polars_df['VIDEO_ID'][0:5], key_format="upper", output_format="pandas")
print(f"Retrieved statistics for {video_stats_polars_df.shape[0]} videos.")
print(f"Remaining units in quota: {client.max_quota - client.quota_used}")
video_stats_polars_df.head()

Retrieved statistics for 5 videos.
Remaining units in quota: 9973


,VIDEO_ID,TITLE,DESCRIPTION,PUBLISHED_AT,CHANNEL_ID,CHANNEL_TITLE,TAGS,CATEGORY_ID,VIEW_COUNT,LIKE_COUNT,COMMENT_COUNT,DURATION_SECONDS,DEFINITION,IS_SHORT,VIDEO_STATS_COMMIT_TIME
0,72Ef65B65JA,Python DeepSeek Tutorial: Installing DeepSeek ...,"In this tutorial, we'll be installing DeepSeek...",2025-02-09T21:01:50Z,UCB2mKxxXPK3X8SJkAc-db3A,CDcodes,[],27,4559,93,18,925,hd,False,2025-03-31 00:43:33.937681
1,S3dlYi7LPk0,Thank You Pygame,"An ode to pygame, the quirky little python mod...",2024-08-18T23:14:38Z,UCB2mKxxXPK3X8SJkAc-db3A,CDcodes,[],27,8748,429,37,400,hd,False,2025-03-31 00:43:33.937681
2,5OoukJNz3YQ,Beginner Python Project From Scratch: Web Scra...,"In this video, I'll be walking you through a r...",2023-05-30T00:05:58Z,UCB2mKxxXPK3X8SJkAc-db3A,CDcodes,[],20,1550,68,11,2737,hd,False,2025-03-31 00:43:33.937681
3,ruISjcokmuk,Beginner Python Project From Scratch: Rock Pap...,Follow along with me on replit: https://replit...,2022-11-09T01:38:55Z,UCB2mKxxXPK3X8SJkAc-db3A,CDcodes,[],20,3323,56,9,960,hd,False,2025-03-31 00:43:33.937681
4,Q6CCdCBVypg,Beginner Python Project From Scratch: Tic Tac Toe,Follow along with me on replit: https://replit...,2022-04-05T22:45:48Z,UCB2mKxxXPK3X8SJkAc-db3A,CDcodes,[],20,78974,811,60,1014,hd,False,2025-03-31 00:43:33.937681


# Get Video Comments

Once you have a list of video IDs, you can the following data regarding each of the comments
+ Comment Text
+ Like counts
+ Reply counts
+ Publication Date

This is useful for performance analytics and sentiment analysis

### Raw output from YouTube API V3

In [26]:
comments = client.get_top_level_comments_for_video_ids(video_ids, key_format="raw", output_format="raw")
print(f"Collected {len(comments)} comments across {len(video_ids)} videos.")
print(f"Remaining units in quota: {client.max_quota - client.quota_used}")
comments[0:2]

Fetching comments for video ID: 72Ef65B65JA
Fetching comments for video ID: S3dlYi7LPk0
Fetching comments for video ID: 5OoukJNz3YQ
Fetching comments for video ID: ruISjcokmuk
Fetching comments for video ID: Q6CCdCBVypg
Collected 72 comments across 5 videos.
Remaining units in quota: 9937


[{'videoId': '72Ef65B65JA',
  'author': '@PARK60264',
  'text': 'Is this local? lama is not local is it?',
  'publishedAt': '2025-03-29T00:12:15Z',
  'likeCount': 0,
  'replyCount': 1,
  'videoTopLevelComments_commit_time': '2025-03-31 00:58:53.192381'},
 {'videoId': '72Ef65B65JA',
  'author': '@xiaoyanzheng6435',
  'text': 'hi thank you for sharing this very helpful! for me it runs a bit slow but still works in the command prompt but when i switched to Jupyter Notebook, it slowed dramatically, a simple question takes 10 minutes for the answer to be produced. Any advice on improving speed? thanks!',
  'publishedAt': '2025-02-27T15:55:04Z',
  'likeCount': 0,
  'replyCount': 1,
  'videoTopLevelComments_commit_time': '2025-03-31 00:58:53.192381'}]

### Pandas output with modified column names

In [24]:
# Get top-level comments from the same 5 videos
comments_pandas_df = client.get_top_level_comments_for_video_ids(videos_pandas_df['video_id'][0:5], key_format="lower", output_format="pandas")
print(f"Collected {comments_pandas_df.shape[0]} comments across {len(videos_pandas_df['video_id'][0:5].unique())} videos.")
print(f"Remaining units in quota: {client.max_quota - client.quota_used}")
comments_pandas_df.tail(5)

Fetching comments for video ID: 72Ef65B65JA
Fetching comments for video ID: S3dlYi7LPk0
Fetching comments for video ID: 5OoukJNz3YQ
Fetching comments for video ID: ruISjcokmuk
Fetching comments for video ID: Q6CCdCBVypg
Collected 72 comments across 5 videos.
Remaining units in quota: 9947


,video_id,author,text,published_at,like_count,reply_count,video_top_level_comments_commit_time
67,Q6CCdCBVypg,@Poxlol,Underrated,2022-04-23T03:21:11Z,2,1,2025-03-31 00:56:00.456982
68,Q6CCdCBVypg,@phoebely2900,Fantastic work!,2022-04-06T04:44:48Z,1,1,2025-03-31 00:56:00.456982
69,Q6CCdCBVypg,@chosenone2435,Great to see you back! You are the best Pygame...,2022-04-06T01:25:19Z,4,1,2025-03-31 00:56:00.456982
70,Q6CCdCBVypg,@joey546,"Good to see you back! You still have, by far, ...",2022-04-05T23:35:12Z,9,2,2025-03-31 00:56:00.456982
71,Q6CCdCBVypg,@MyFishWillKillU,Wow so cool,2022-04-05T22:53:18Z,4,0,2025-03-31 00:56:00.456982


### Polars output with modified column names

In [25]:
# Get top-level comments from the same 5 videos
comments_polars_df = client.get_top_level_comments_for_video_ids(videos_polars_df['VIDEO_ID'][0:5], key_format="upper", output_format="polars")
print(f"Collected {comments_polars_df.shape[0]} comments across {len(comments_polars_df['VIDEO_ID'][0:5].unique())} videos.")
print(f"Remaining units in quota: {client.max_quota - client.quota_used}")
comments_polars_df.tail(5)

Fetching comments for video ID: 72Ef65B65JA
Fetching comments for video ID: S3dlYi7LPk0
Fetching comments for video ID: 5OoukJNz3YQ
Fetching comments for video ID: ruISjcokmuk
Fetching comments for video ID: Q6CCdCBVypg
Collected 72 comments across 1 videos.
Remaining units in quota: 9942


VIDEO_ID,AUTHOR,TEXT,PUBLISHED_AT,LIKE_COUNT,REPLY_COUNT,VIDEO_TOP_LEVEL_COMMENTS_COMMIT_TIME
str,str,str,str,i64,i64,str
"""Q6CCdCBVypg""","""@Poxlol""","""Underrated""","""2022-04-23T03:21:11Z""",2,1,"""2025-03-31 00:56:46.682534"""
"""Q6CCdCBVypg""","""@phoebely2900""","""Fantastic work!""","""2022-04-06T04:44:48Z""",1,1,"""2025-03-31 00:56:46.682534"""
"""Q6CCdCBVypg""","""@chosenone2435""","""Great to see you back! You are…","""2022-04-06T01:25:19Z""",4,1,"""2025-03-31 00:56:46.682534"""
"""Q6CCdCBVypg""","""@joey546""","""Good to see you back! You stil…","""2022-04-05T23:35:12Z""",9,2,"""2025-03-31 00:56:46.682534"""
"""Q6CCdCBVypg""","""@MyFishWillKillU""","""Wow so cool""","""2022-04-05T22:53:18Z""",4,0,"""2025-03-31 00:56:46.682534"""
